In [1]:
import sys
import os
from autogen import ConversableAgent

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(".."))

from llm_config import llm_config

In [ ]:
job_advisor = ConversableAgent(
     name="Job_Advisor",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1
)

role_suggester = ConversableAgent(
    name="Role_Suggester",
    system_message="You are an expert in career coaching. Suggest 2-3 job roles based on user's skills.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

market_analyst = ConversableAgent(
    name="Market_Analyst",
    system_message="You analyze job market trends. For a given role, respond with 'High', 'Moderate', or 'Low' demand with a 1-line reason.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

salary_estimator = ConversableAgent(
    name="Salary_Estimator",
    system_message="Estimate an average annual salary for a given job role in USD. Respond in one sentence.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

In [ ]:
nested_chats = [
    {
        "recipient": role_suggester,
        "message": "Suggest 2-3 job roles based on my skills in Python, data analysis, and machine learning.",
        "summary_method": "reflection_with_llm",
        "max_turns": 1
    },
    {
        "recipient": market_analyst,
        "message": "Analyze the market demand for those job roles.",
        "summary_method": "reflection_with_llm",
        "max_turns": 1
    },
    {
        "recipient": salary_estimator,
        "message": "Estimate the average salary for those job roles.",
        "summary_method": "reflection_with_llm",
        "max_turns": 1
    },
]


In [14]:
# Link nested logic to the Job Advisor
job_advisor.register_nested_chats(
    nested_chats,
    trigger=lambda sender: sender is None or sender.name not in ["Role_Suggester", "Market_Analyst", "Salary_Estimator"]
)

In [ ]:
# 🧪 Try with user input
reply = job_advisor.generate_reply(
    messages=[{"role": "user", "content": "I have skills in Python, data analysis, and machine learning. What job should I go for?"}]
)

# ✅ Print final reply (aggregated from nested agents)
print("\n💬 Final Recommendation:\n", reply)


********************************************************************************
Starting a new chat....

********************************************************************************
Job_Advisor (to Role_Suggester):

Suggest 2–3 job roles based on my skills in Python, data analysis, and machine learning.

--------------------------------------------------------------------------------
Role_Suggester (to Job_Advisor):

Given your skills in Python, data analysis, and machine learning, here are a few job roles that might be suitable for you:

1. **Data Scientist**: As a Data Scientist, you'll leverage your knowledge of Python and machine learning algorithms to analyze complex data and generate actionable insights. This role often involves building predictive models and machine learning systems to solve various industry-specific problems.

2. **Machine Learning Engineer**: This role focuses more on building and deploying machine learning applications as opposed to the broader explorat

In [4]:
import os
import sys
from autogen import ConversableAgent

sys.path.append(os.path.abspath(".."))
from llm_config import llm_config

job_advisor = ConversableAgent(
                "job_adviser",
                system_message = "You are an assistant agent providing job adivise in less than 20 words.",
                llm_config = llm_config,
                human_input_mode = "NEVER",
                max_consecutive_auto_reply = 15
)

job_role_advisor = ConversableAgent(
                "job_role_advisor",
                system_message = "You are an assistant agent providing job roles based on the skills. Explain in less than 20 words.",
                llm_config = llm_config,
                human_input_mode = "NEVER"
)

market_analyst = ConversableAgent(
                "market_analyst",
                system_message = "For the given job roles, do a market analysis and categorize job roles as high, mid and low in demands."
                "Provide response in less than 20 words.",
                llm_config = llm_config,
                human_input_mode = "NEVER"
)

salary_estimator = ConversableAgent(
                "salary_estimator",
                system_message = "Provide yearly estimated salary of the given job roles in INR. Provide response in less than 20 words.",
                llm_config = llm_config,
                human_input_mode = "NEVER"
)

nested_chats = [
                {
                    "recipient" : job_role_advisor,
                    "message" : "Provide job roles for Data Science, Machine Learning, Computer Vision and NLP.",
                    "summary_method" : "reflection_with_llm",
                    "max_turns" : 2
                },
                {
                    "recipient" : market_analyst,
                    "message" : "Advise current market for the given job roles.",
                    "summary_method" : "reflection_with_llm",
                    "max_turns" : 2
                },
                {
                    "recipient" : salary_estimator,
                    "message" : "Estimate salary for the given job roles.",
                    "summary_method" : "reflection_with_llm",
                    "max_turns" : 2
                }
]

job_advisor.register_nested_chats(
                nested_chats,
                trigger = lambda agnt : agnt is None or agnt.name not in ["job_role_advisor", "market_analyst", "salary_estimator"]
)

reply = job_advisor.generate_reply(
            messages = [{"role" : "user", "content" : "Advise job roles for the given skills."}]
)


********************************************************************************
Starting a new chat....

********************************************************************************
job_adviser (to job_role_advisor):

Provide job roles for Data Science, Machine Learning, Computer Vision and NLP.

--------------------------------------------------------------------------------
job_role_advisor (to job_adviser):

Data Scientist, Machine Learning Engineer, Computer Vision Engineer, NLP Research Scientist.

--------------------------------------------------------------------------------
job_adviser (to job_role_advisor):

Consider Data Analyst, AI Engineer, Research Scientist, or ML Ops Engineer for diverse opportunities.

--------------------------------------------------------------------------------
job_role_advisor (to job_adviser):

Data Analyst, AI Engineer, Research Scientist, ML Ops Engineer offer diverse roles in data-driven and AI-focused environments.

--------------------

In the above code, each time the job_advisor will call the agents in the nested chat one by one sequentially. Each of the agents will be called back to back max_turns times. If the max_turns exceeded, the chat will be terminated and the next chat will be starting with the next agent in the nested_chat by using the chat history saved by summary_method. It repeats until either the job_advisor runs max_consecutive_auto_reply or if all the agents in the nested_chat completed execution max_turns times, whichever is earlier.